In [ ]:
import networkx as nx
import logging
import multiprocessing as mp
import pandas as pd
import sys
log = logging.getLogger("regulatory")
log.setLevel(logging.DEBUG)

http://www.regnetworkweb.org/source.jsp

In [ ]:
g = nx.Graph()

In [ ]:
def process_ENCODE_row(row):
    miRNA, gene = row.replace("\n", "").split("\t")[:2]
    if "M" not in miRNA: #miRNA regulate genes, first edge should be mirna
        return None
    else:
        return (miRNA, gene)
    
with open("human/ENCODE", "r") as file:
    pool = mp.Pool()
    w = pool.map_async(process_ENCODE_row, file.readlines(), error_callback=lambda err: log.error(err))
    pool.close()
    
pool.join()

g.add_edges_from([wi for wi in w.get() if wi is not None])

In [ ]:
nodes = g.nodes()

def process_row(row):
        node = row.replace("\n", "").split("\t")
        if node[0] in nodes:
            return node[0], {"name":node[1].replace("hsa-miR", "hsa-mir")}
        else:
            return None
        
with open("human/human.node", "r") as file:
    pool = mp.Pool()
    w = pool.map_async(process_row, file.readlines(), error_callback=lambda err: log.error(err))
    pool.close()
    
pool.join()
g.add_nodes_from([wi for wi in w.get() if wi is not None])

In [ ]:
df_adjiacency = pd.DataFrame(g.nodes(data=True), columns = ["id", "data"]).set_index("id")
def get_name(id):
    try:
        return df_adjiacency.loc[id,"data"]["name"]
    except:
        return None
df_adjiacency["name"] = list(map(get_name, df_adjiacency.index))

def get_name(id):
    try:
        return df_adjiacency.at[id,"name"]
    except:
        return None

In [ ]:
df_adjiacency.head()

In [ ]:
for gene in list(map(get_name,g.neighbors("MIMAT0002882"))):
    print(gene)

In [ ]:
nx.write_gpickle(g, "graph.pkl.gz")

In [ ]:
df_adjiacency.reset_index().to_csv("graph_metadata.csv")